In [1]:
# Nodos hoja
e1 = dict ( tipo = "evento" , nombre = "E1" , prob =0.2 , hijos =[])
e2 = dict ( tipo = "evento" , nombre = "E2" , prob =0.3 , hijos =[])
e3 = dict ( tipo = "evento" , nombre = "E3" , prob =0.3 , hijos =[])
# Puerta OR
e1e2 = dict ( tipo = "OR" , nombre = None , prob = None , hijos =[ e1 , e2 ])
# Evento intermedio
e4 = dict ( tipo = "evento" , nombre = "E4" , prob = None , hijos =[ e1e2 ])
# Puerta AND
e4e3 = dict ( tipo = "AND" , nombre = None , prob = None , hijos =[ e4 , e3 ])
# Nodo final
af = dict ( tipo = "evento" , nombre = "E5" , prob = None , hijos =[ e4e3 ])
arbol = {
    "e1": e1,
    "e2": e2,
    "e3": e3,
    "e1e2": e1e2,
    "e4": e4,
    "e4e3": e4e3,
    "af": af
}

In [2]:
# Ejercicio 1
def buscar_nodo(nodo : dict, nombre : str) -> list[dict]:
    lista_nodos = []
    for value in nodo:
        if value == nombre:
            lista_nodos.append(value)
            return lista_nodos

print(buscar_nodo(arbol, "e1"))


['e1']


In [12]:
# Ejercicio 2
def propagacion(nodo: dict) -> None:
    for key in nodo:
        if nodo[key]["tipo"] == "evento":
            # Si el nodo es un evento y su probabilidad es None, pero tiene hijos
            if nodo[key]["prob"] is None and nodo[key]["hijos"]:
                hijo = nodo[key]["hijos"][0]
                # Si el hijo es una puerta lógica y tiene una probabilidad definida
                if hijo["tipo"] in ["OR", "AND"] and hijo["prob"] is not None:
                    nodo[key]["prob"] = hijo["prob"]
            print(f"{nodo[key]['nombre']}: {nodo[key]['prob']}")
        elif nodo[key]["tipo"] == "OR":
            # Calcular la probabilidad para la puerta OR
            nodo[key]["prob"] = 1 - (1 - nodo[key]["hijos"][0]["prob"]) * (1 - nodo[key]["hijos"][1]["prob"])
        elif nodo[key]["tipo"] == "AND":
            # Calcular la probabilidad para la puerta AND
            nodo[key]["prob"] = nodo[key]["hijos"][0]["prob"] * nodo[key]["hijos"][1]["prob"]
        else:
            print("Error")

propagacion(arbol)

Procesador1: 0.15351827510938587
Procesador2: 0.15351827510938587
DiscoDuro: 0.28346868942621073
TarjetaRed1: 0.18126924692201818
TarjetaRed2: 0.18126924692201818
Fuente1: 0.22119921692859512
Fuente2: 0.22119921692859512
Servidor: 0.30035579960732517


In [4]:
# Ejercicio 3
def nodos(nodo:dict) -> list:
    lista_nodos = []
    for clave in nodo:
        if nodo[clave]["tipo"] in ["OR", "AND"]:
            lista_nodos.append(nodo[clave]["tipo"])
        else:
            lista_nodos.append(nodo[clave]["nombre"])
    return lista_nodos
print(nodos(arbol))

['E1', 'E2', 'E3', 'OR', 'E4', 'AND', 'E5']


In [5]:
# Ejercicio 4
def eventos(nodo:dict) -> list:
    lista_nodos = []
    for clave in nodo:
        if nodo[clave]["tipo"] == "evento":
            lista_nodos.append(nodo[clave]["nombre"])
    return lista_nodos
    
print(eventos(arbol))

['E1', 'E2', 'E3', 'E4', 'E5']


In [6]:
# Ejercicio 5
def evento_info(nodo:dict, nombre:str) -> tuple:
    for key, value in nodo.items():
        if key == nombre:
            if value["tipo"] in ["OR", "AND"] and len(value["hijos"]) == 2:
                return (value["tipo"], value["hijos"][0]["nombre"], value["hijos"][1]["nombre"])
    return None

print(evento_info(arbol, "e1e2"))
print(evento_info(arbol, "e4e3"))


('OR', 'E1', 'E2')
('AND', 'E4', 'E3')


In [10]:
# Ejercicio 6
from pyAgrum import BayesNet
from pyAgrum import LazyPropagation
import itertools

# Función para transformar un nodo en una red bayesiana
def transformar(nodo: dict):
    red_bayes = BayesNet()
    added_nodes = set()  # Conjunto para rastrear nodos añadidos y evitar duplicados

    # Función interna para agregar nodos a la red bayesiana
    def agregar_nodo(nodo, padre=None):
        if nodo["tipo"] == "evento":
            nombre_evento = nodo["nombre"]
            if nombre_evento not in added_nodes:  # Verificar si el nodo ya fue añadido
                red_bayes.add(nombre_evento)
                added_nodes.add(nombre_evento)  # Marcar como añadido
                if nodo["prob"] is not None:
                    red_bayes.cpt(nombre_evento).fillWith([1 - nodo["prob"], nodo["prob"]])
            if padre is not None:
                if not red_bayes.existsArc(nombre_evento, padre):
                    red_bayes.addArc(nombre_evento, padre)
        elif nodo["tipo"] in ["AND", "OR"]:
            nombre_evento = "Nodo_" + str(id(nodo))
            if nombre_evento not in added_nodes:  # Verificar si el nodo ya fue añadido
                red_bayes.add(nombre_evento)
                added_nodes.add(nombre_evento)  # Marcar como añadido
        nombres_hijos = []      

        for hijo in nodo["hijos"]:
            agregar_nodo(hijo, nombre_evento)  # Agregar nodos hijos
            nombres_hijos.append(hijo["nombre"])

        # Configurar la CPT del nodo compuesto
        if nodo["tipo"] == "AND":
            for values in itertools.product([0, 1], repeat=len(nombres_hijos)):
                prob = 1 if all(values) else 0
                red_bayes.cpt(nombre_evento)[dict(zip(nombres_hijos, values))] = [1 - prob, prob]
        elif nodo["tipo"] == "OR":
            for values in itertools.product([0, 1], repeat=len(nombres_hijos)):
                prob = 1 if any(values) else 0
                red_bayes.cpt(nombre_evento)[dict(zip(nombres_hijos, values))] = [1 - prob, prob]

    agregar_nodo(nodo)
    return red_bayes

# Transformar el nodo af en una red bayesiana
red_bayes = transformar(af)
print(red_bayes)

# Realizar inferencia en la red bayesiana
inferencia = LazyPropagation(red_bayes)
inferencia.makeInference()

# Obtener la probabilidad posterior del evento E5
probabilidad_E5 = inferencia.posterior("E5")
print(f"Inferencia de E5: {probabilidad_E5}")


BN{nodes: 7, arcs: 4, domainSize: 128, dim: 13, mem: 208o}
Inferencia de E5: 
  E5               |
0        |1        |
---------|---------|
 0.8680  | 0.1320  |



In [8]:
import math
def prob_fallo(mttf, t=1):
    return 1 - math.exp(-t / mttf)

# Tiempos medios de vida (en años)
mttf_procesadores = 6
mttf_disco_duro = 3
mttf_tarjetas_red = 5
mttf_fuentes = 4

# Nodos de eventos con probabilidades calculadas
procesador1 = dict(tipo="evento", nombre="Procesador1", prob=prob_fallo(mttf_procesadores), hijos=[])
procesador2 = dict(tipo="evento", nombre="Procesador2", prob=prob_fallo(mttf_procesadores), hijos=[])
disco_duro = dict(tipo="evento", nombre="DiscoDuro", prob=prob_fallo(mttf_disco_duro), hijos=[])
tarjeta_red1 = dict(tipo="evento", nombre="TarjetaRed1", prob=prob_fallo(mttf_tarjetas_red), hijos=[])
tarjeta_red2 = dict(tipo="evento", nombre="TarjetaRed2", prob=prob_fallo(mttf_tarjetas_red), hijos=[])
fuente1 = dict(tipo="evento", nombre="Fuente1", prob=prob_fallo(mttf_fuentes), hijos=[])
fuente2 = dict(tipo="evento", nombre="Fuente2", prob=prob_fallo(mttf_fuentes), hijos=[])

# Nodos de puertas lógicas
procesadores_fallo = dict(tipo="AND", nombre="Procesadores_fallo", prob=None, hijos=[procesador1, procesador2])
tarjetas_red_fallo = dict(tipo="AND", nombre="Tarjetas_red_fallo", prob=None, hijos=[tarjeta_red1, tarjeta_red2])
fuentes_fallo = dict(tipo="AND", nombre="Fuentes_fallo", prob=None, hijos=[fuente1, fuente2])

# Nodo raíz que representa el fallo del sistema
servidor_fallo = dict(tipo="OR", nombre="Servidor_fallo", prob=None, hijos=[procesadores_fallo, disco_duro, tarjetas_red_fallo, fuentes_fallo])

af_servidor_fallo = dict(tipo="evento", nombre="Servidor", prob=None, hijos=[servidor_fallo])


In [9]:
arbol = {
    "procesador1": procesador1,
    "procesador2": procesador2,
    "disco_duro": disco_duro,
    "tarjeta_red1": tarjeta_red1,
    "tarjeta_red2": tarjeta_red2,
    "fuente1": fuente1,
    "fuente2": fuente2,
    "procesadores_fallo": procesadores_fallo,
    "tarjetas_red_fallo": tarjetas_red_fallo,
    "fuentes_fallo": fuentes_fallo,
    "servidor_fallo": servidor_fallo,
    "af_servidor_fallo": af_servidor_fallo
}

# Ejercicio 1
print(buscar_nodo(arbol, "fuente1"))
# Ejercicio 2
propagacion(arbol)
# Ejercicio 3
print(nodos(arbol))
# Ejercicio 4
print(eventos(arbol))
# Ejercicio 5
print(evento_info(arbol, "procesadores_fallo"))
print(evento_info(arbol, "tarjetas_red_fallo"))
# Ejercicio 6
red_bayesiana = transformar(af_servidor_fallo)
print(red_bayesiana)
inferencia = LazyPropagation(red_bayesiana)
inferencia.makeInference()
probabilidad_Servidor = inferencia.posterior("Servidor")
print(f"Inferencia del Servidor: {probabilidad_Servidor}")

['fuente1']
Procesador1: 0.15351827510938587
Procesador2: 0.15351827510938587
DiscoDuro: 0.28346868942621073
TarjetaRed1: 0.18126924692201818
TarjetaRed2: 0.18126924692201818
Fuente1: 0.22119921692859512
Fuente2: 0.22119921692859512
Servidor: 0.30035579960732517
['Procesador1', 'Procesador2', 'DiscoDuro', 'TarjetaRed1', 'TarjetaRed2', 'Fuente1', 'Fuente2', 'AND', 'AND', 'AND', 'OR', 'Servidor']
['Procesador1', 'Procesador2', 'DiscoDuro', 'TarjetaRed1', 'TarjetaRed2', 'Fuente1', 'Fuente2', 'Servidor']
('AND', 'Procesador1', 'Procesador2')
('AND', 'TarjetaRed1', 'TarjetaRed2')
Agregando nodo de evento: Servidor
Agregando nodo compuesto: Nodo_2258791506624
Agregando nodo compuesto: Nodo_2258791506944
Agregando nodo de evento: Procesador1
Agregando arco de Procesador1 a Nodo_2258791506944
Agregando nodo de evento: Procesador2
Agregando arco de Procesador2 a Nodo_2258791506944
Agregando nodo de evento: DiscoDuro
Agregando arco de DiscoDuro a Nodo_2258791506624
Agregando nodo compuesto: Nodo